In [1]:
from auditory_cortex.Regression import transformer_regression
from auditory_cortex.ridge_regression import RidgeRegression 


import sys
import yaml
from yaml import Loader
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
# from sklearn.model_selection import KFold
import time
import csv


conf_path = '/scratch/gilbreth/akamsali/Research/Makin/Auditory_Cortex/conf/ridge_conf.yaml'

# conf_path = '/Users/akshita/Documents/Research/Makin/Auditory_Cortex/conf/ridge_conf.yaml'
with open(conf_path, "r") as f:
    manifest = yaml.load(f, Loader=Loader)

data_path = manifest['data_path']
subject = manifest['sub']
output_dir = manifest['output_dir']

# data_path = '/Users/akshita/Documents/Research/Makin/data'
# subject = '200206'
# output_dir = '/Users/akshita/Documents/Research/Makin/Auditory_Cortex'

reg = transformer_regression(data_path, subject)

test_list = np.arange(450,499).tolist()
# train_val_list = np.arange(1,450)
train_list = np.arange(1,450).tolist()
w = 80
sp = 1
num_layers = len(reg.layers)

# alpha = 1
# for l in range(num_layers):
l = 0

k_val_test , def_w_test, offset_test, z_vals_test = reg.get_layer_values(l, win=w, sent_list=test_list)
n_vals_test = reg.get_all_channels(def_w=def_w_test, offset=offset_test, k_val=k_val_test, sent_list=test_list)

k_val_train , def_w_train, offset_train, z_vals_train = reg.get_layer_values(layer=l, win=w, sent_list=train_list)
n_vals_train = reg.get_all_channels(def_w=def_w_train, offset=offset_train, k_val=k_val_train, sent_list=train_list)

ridge_model_sk = LinearRegression()
ridge_model_sk.fit(z_vals_train, n_vals_train)
y_hat_train = ridge_model_sk.predict(z_vals_train)
# y_hat_val = ridge_model.predict(x_val)
y_hat_test = ridge_model_sk.predict(n_vals_test)

# ridge_model = RidgeRegression(alpha=alpha)
# ridge_model.fit(z_vals_train, n_vals_train)
# n_hat_train = ridge_model.predict(z_vals_train)
# n_hat_test = ridge_model.predict(z_vals_test)

print(f"train: {r2_score(n_vals_train, y_hat_train)}, test: {r2_score(n_vals_test, y_hat_test)}")

/home/akamsali/anaconda3/envs/research_env/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630815121/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


KeyboardInterrupt: 